In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [2]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [3]:
%matplotlib notebook

In [4]:
df = pd.read_excel('赛题1测试数据集\购地-房地产大企业购地情况.xlsx', index_col='企业编号')
df

,行政区,签订日期,供地总面积（公顷）,约定动工时间,土地用途,供应方式,最小容积率,最大容积率,成交价款（万元）,约定竣工时间
企业编号,,,,,,,,,,
4235,杭州市,2013-09-27,0.746100,NaN,工业用地,NaN,NaN,NaN,360.00000,NaN
4236,安徽省 > 淮南市 > 淮南市本级,2011-12-30,7.141500,NaN,工业用地,NaN,NaN,NaN,2406.69000,NaN
4237,江西省 > 南昌市 > 南昌县,2012-06-28,0.674700,NaN,工业用地,NaN,NaN,NaN,345.67000,NaN
4238,广东省 > 深圳市 > 深圳市本级,2018-07-04,0.593388,NaN,工业用地,NaN,NaN,NaN,27100.00000,NaN
4242,广东省 > 珠海市 > 珠海市本级,2012-05-18,1.146056,NaN,工业用地,NaN,NaN,NaN,550.10688,NaN
...,...,...,...,...,...,...,...,...,...,...
4520,吴江市,2016-07-19,4.187240,NaN,工业用地,NaN,NaN,NaN,1415.28712,NaN
4522,江苏省 > 南通市 > 如东县,2017-10-30,3.375900,NaN,工业用地,NaN,NaN,NaN,962.13150,NaN
4523,中山市,2014-07-23,5.650500,NaN,工矿仓储用地,NaN,NaN,NaN,NaN,NaN


In [5]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [6]:
appointment_time = df[['约定动工时间']].applymap(to_day).groupby('企业编号').mean().T
appointment_time = appointment_time.reindex(columns=list(range(4001,4526)))
appointment_time

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
约定动工时间,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-562.0,NaN,NaN,NaN,NaN,-47.0,NaN


In [7]:
dt = df[['约定竣工时间']].applymap(to_day).groupby('企业编号').mean().T
dt = dt.reindex(columns=list(range(4001,4526)))
dt


企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
约定竣工时间,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def limit(cell):
    if cell > 1:
        return 1
    return cell

In [9]:
dummy_1=pd.get_dummies(df["供应方式"],prefix="供应方式")
dummy_1=dummy_1.groupby("企业编号").sum().T
dummy_1=dummy_1.applymap(limit).reindex(columns=list(range(4001,4526))).fillna(0).reindex(['供应方式_划拨'
               , '供应方式_协议出让', '供应方式_拍卖出让', '供应方式_招标出让', '供应方式_挂牌出让']).fillna(0)
dummy_1

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
供应方式_划拨,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
供应方式_协议出让,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
供应方式_拍卖出让,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
供应方式_招标出让,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
供应方式_挂牌出让,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
m=df.groupby(["企业编号"]).sum().drop(columns=["供地总面积（公顷）","成交价款（万元）","约定竣工时间"]).T.reindex(columns=list(range(4001,4526)))
m

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
最小容积率,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
最大容积率,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN


In [11]:
appointment_time.append([dt,dummy_1,m]).to_pickle('购地-房地产大企业购地情况-testing.pickle')